<a href="https://www.kaggle.com/code/shariq20220/cv-project-arcfaceloss?scriptVersionId=191291355" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%pip install wildlife-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install wildlife-tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install timm

Note: you may need to restart the kernel to use updated packages.


In [4]:
import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

In [5]:
import itertools
from itertools import chain
import timm
import pandas as pd
import numpy as np
import torch
import torchvision.transforms as T
from torch.optim import SGD

from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset

from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.train import ArcFaceLoss, BasicTrainer, TripletLoss

2024-08-05 14:28:13.430489: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 14:28:13.430585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 14:28:13.560171: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**IPanda50 Dataset**

In [6]:
# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()


DATASET IPanda50: DOWNLOADING STARTED.


Downloading...
From (original): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg
From (redirected): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg&confirm=t&uuid=06f6d457-2994-4b5c-80a8-d5adade8952d
To: /kaggle/data/IPanda50/iPanda50-images.zip
100%|██████████| 926M/926M [00:07<00:00, 125MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gVREtFWkNec4xwqOyKkpuIQIyWU_Y_Ob
To: /kaggle/data/IPanda50/iPanda50-split.zip
100%|██████████| 140k/140k [00:00<00:00, 78.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jdACN98uOxedZDT-6X3rpbooLAAUEbNY
To: /kaggle/data/IPanda50/iPanda50-eyes-labels.zip
100%|██████████| 2.55M/2.55M [00:00<00:00, 142MB/s]


DATASET IPanda50: EXTRACTING STARTED.
DATASET IPanda50: FINISHED.



config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/204M [00:00<?, ?B/s]

...

Epoch 18: Average Loss = 4.570041656494141


Epoch 18: 100%|█████████████████████████████████████████████████████| 54/54 [01:23<00:00,  1.55s/it]


Epoch 19: Average Loss = 3.812040090560913


Epoch 19: 100%|█████████████████████████████████████████████████████| 54/54 [01:24<00:00,  1.56s/it]

Epoch 20: Average Loss = 3.461237907409668


In [7]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_IPanda50_retrained_checkpoint.pth")

In [8]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [9]:
# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone)
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 53/53 [14:56<00:00, 16.92s/it]


In [10]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[ 4.62737773e-03, -1.27493246e-02,  5.70487045e-02, ...,
         9.45716143e-01, -5.09465151e-02,  3.45591940e-02],
       [ 6.14794418e-02, -8.61518085e-02,  3.50631997e-02, ...,
         5.05064875e-02, -9.51805562e-02,  1.54856965e-02],
       [ 4.18375060e-02,  2.17879266e-02,  8.85178149e-02, ...,
        -1.10139273e-01,  5.57891876e-02, -5.64030185e-02],
       ...,
       [-6.38543740e-02, -2.96911784e-02,  2.07411274e-02, ...,
         5.99760562e-02, -2.93768123e-02, -3.43887992e-02],
       [-4.28346172e-02,  2.95903534e-02, -7.80144259e-02, ...,
        -7.51395598e-02,  4.48252074e-04, -6.35410547e-02],
       [ 8.71770531e-02,  1.71552505e-02,  8.87418747e-01, ...,
         3.07812467e-02, -1.67433135e-02, -7.47750103e-02]], dtype=float32)}


In [11]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['36_xingda' '37_xinger' '01_aoliao' '33_wuyi' '47_yingying' '15_maosun'
 '09_fushun' '10_hexing' '36_xingda' '19_nannan' '08_fulai' '31_shurong'
 '44_yayi' '39_xinghui' '28_sa' '14_maodou' '14_maodou' '00_aibang'
 '21_nina' '35_xilan' '19_nannan' '33_wuyi' '47_yingying' '49_yuanrun'
 '36_xingda' '34_xiaoqiao' '16_maotao' '48_yongbang' '11_jiaoao'
 '49_yuanrun' '36_xingda' '28_sa' '35_xilan' '35_xilan' '02_baolan'
 '31_shurong' '31_shurong' '02_baolan' '23_qiubang' '26_qiyuan'
 '04_chengdui' '35_xilan' '08_fulai' '00_aibang' '28_sa' '29_shuangxiong'
 '46_yazhu' '44_yayi' '14_maodou' '30_shuqing' '36_xingda' '39_xinghui'
 '33_wuyi' '10_hexing' '10_hexing' '00_aibang' '49_yuanrun' '35_xilan'
 '04_chengdui' '10_hexing' '39_xinghui' '36_xingda' '37_xinger'
 '36_xingda' '16_maotao' '22_nini' '00_aibang' '39_xinghui' '10_hexing'
 '17_meibang' '00_aibang' '28_sa' '14_maodou' '15_maosun' '32_susu'
 '26_qiyuan' '27_rourou' '23_qiubang' '18_miaomiao' '48_yongba

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [12]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on IPanda50 data: {:.2f}%".format(accuracy_P * 100))

Accuracy on IPanda50 data: 100.00%


In [13]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [14]:
# # Assuming you need to use the model for further operations
# # Initialize the model architecture again as needed for loading
# # Make sure to initialize it exactly as you did for training
# model = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=False)

# # # Create a BasicTrainer instance with the initialized model (other parameters should match the training setup)
# # trainer = BasicTrainer(
# #     dataset=None,  # Assuming no dataset is needed just for loading
# #     model=model,
# #     objective=None,  # Assuming no objective needed just for loading
# #     optimizer=None,  # Assuming no optimizer needed just for loading
# #     epochs=0,  # No training epochs needed for just loading
# #     device='cuda'  # Adjust as per your device configuration
# # )

# trainer = BasicTrainer(
#     dataset=dataset,
#     model=backbone,
#     objective=objective,
#     optimizer=optimizer,
#     epochs=2,
#     device='cuda',
#     epoch_callback=print_epoch_loss
# )
# # Load the model
# trainer.load("retrained_chks/arcfaceloass_retrained_checkpoint.pth")

# # After loading, the model in the trainer instance is now the re-trained model
# # and is ready for use in further processing or inference


**MacaqueFaces Dataset**

In [15]:
# Download dataset (if not already downloaded)
datasets.MacaqueFaces.get_data('../data/MacaqueFaces')
# Load dataset metadata
metadata = datasets.MacaqueFaces('../data/MacaqueFaces')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET MacaqueFaces: DOWNLOADING STARTED.


MacaqueFaces.zip: 12.0MB [00:01, 10.6MB/s]                            
MacaqueFaces_ImageInfo.csv: 410kB [00:00, 850kB/s]                             


DATASET MacaqueFaces: EXTRACTING STARTED.
DATASET MacaqueFaces: FINISHED.



Epoch 0:   0%|                                                               | 0/50 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  98%|████████████████████████████████████████████████████▉ | 49/50 [01:14<00:01,  1.58s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 50/50 [01:14<00:00,  1.50s/it]


Epoch 1: Average Loss = 24.314781188964844


...

Epoch 18: Average Loss = 0.4758913815021515


Epoch 18: 100%|█████████████████████████████████████████████████████| 50/50 [01:16<00:00,  1.53s/it]


Epoch 19: Average Loss = 0.4763692617416382


Epoch 19: 100%|█████████████████████████████████████████████████████| 50/50 [01:16<00:00,  1.53s/it]

Epoch 20: Average Loss = 0.4605986773967743


In [16]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_MacaqueFaces_retrained_checkpoint.pth")

In [17]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone)
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 49/49 [11:19<00:00, 13.87s/it]


In [18]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[0.95468915, 0.9814916 , 0.97475797, ..., 0.03484871, 0.03317138,
        0.03485792],
       [0.9511421 , 0.9833151 , 0.97526795, ..., 0.04288536, 0.04193668,
        0.04381046],
       [0.9523467 , 0.987751  , 0.98488176, ..., 0.04837301, 0.04846193,
        0.04922565],
       ...,
       [0.9485157 , 0.99161106, 0.98612964, ..., 0.04027817, 0.03808139,
        0.04073986],
       [0.9469    , 0.98432803, 0.9787035 , ..., 0.02632249, 0.02223955,
        0.02529176],
       [0.962025  , 0.9832096 , 0.97767526, ..., 0.07197577, 0.07143133,
        0.07235464]], dtype=float32)}


In [19]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [20]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy_P * 100))

Accuracy on MacaqueFaces data: 100.00%


In [21]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**LionData Dateset**

In [22]:
# Download dataset (if not already downloaded)
datasets.LionData.get_data('../data/LionData')
# Load dataset metadata
metadata = datasets.LionData('../data/LionData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET LionData: DOWNLOADING STARTED.


main.zip: 495MB [00:20, 24.5MB/s]


DATASET LionData: EXTRACTING STARTED.
DATASET LionData: FINISHED.



Epoch 0:   0%|                                                                | 0/6 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|████████████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.37s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|████████████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.49s/it]


Epoch 1: Average Loss = 36.7537841796875


...

Epoch 18: Average Loss = 33.59912109375


Epoch 18: 100%|███████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.57s/it]


Epoch 19: Average Loss = 33.2831916809082


Epoch 19: 100%|███████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.57s/it]

Epoch 20: Average Loss = 33.0254020690918


In [23]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_LionData_retrained_checkpoint.pth")

In [24]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [25]:
# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone)
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|█████████████████████████████████████████████████████████████████| 5/5 [01:13<00:00, 14.78s/it]


In [26]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[0.44797313, 0.5143164 , 0.6943141 , ..., 0.6707902 , 0.73745245,
        0.67614806],
       [0.6575196 , 0.41256887, 0.7771603 , ..., 0.38197142, 0.7956885 ,
        0.5503359 ],
       [0.28929207, 0.4202712 , 0.5508639 , ..., 0.5799427 , 0.66377246,
        0.75224835],
       ...,
       [0.36464602, 0.67755586, 0.63303363, ..., 0.7349057 , 0.6634381 ,
        0.64576685],
       [0.29454154, 0.44425255, 0.616675  , ..., 0.5823014 , 0.73368835,
        0.8452034 ],
       [0.32611305, 0.5021293 , 0.6098867 , ..., 0.62401533, 0.69442487,
        0.775141  ]], dtype=float32)}


In [27]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['Enadalut' 'Saba' 'Siena' 'Naini' 'Tikki' 'Moswen' 'Enderoni' 'Simaloi'
 'Spring' 'Tigisi' 'Ngare' 'Chaimu' 'Sangiki' 'Ngare' 'Sikio-Kali'
 'Blackie' 'Sadala' 'Etana' 'Enderoni' 'Kinna' 'Saruni' 'Kaka' 'Safi'
 'Olbarnoti' 'Kibibi' 'Opi' 'Ayo' 'Selenkay' 'Sadala' 'Ngare' 'Moswen'
 'Napi' 'Siti' 'Simaloi' 'Napi' 'Manzy' 'Sero' 'Kioni' 'Kioni' 'Mashavu'
 'Samir' 'Kinna' 'Shemsa' 'Sebastian' 'Lemuanik' 'Naisiae' 'Nguro'
 'Rescue' 'Kibibi' 'Naini' 'Kioni' 'Lucinda' 'Kinna' 'Ngare' 'Sadala'
 'Mashavu' 'Kioni' 'Nguro' 'Engiyaa' 'Saimutie' 'Nashipai' 'Sikio-Kali'
 'Ngare' 'Kioni' 'Nasieku' 'Sero' 'Nakato' 'Enadalut' 'Chaimu' 'Kinna'
 'Nasieku' 'Olonyori' 'Sero' 'Esiriwua' 'Tigisi' 'Nasieku' 'Autumn'
 'Chaimu' 'Lemayian' 'Nariku-Inkgera' 'Enderoni' 'White-Eye' 'Mama-Kali'
 'Moswen' 'Lucinda' 'Naini' 'Sebastian' 'Jamal' 'Empurra' 'Enadalut'
 'Saitoti' 'Naimina' 'Napono' 'Leia' 'Selenkay' 'Senteu' 'Maskio' 'Enkume'
 'Engiyaa' 'Etana']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [28]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on LionData data: {:.2f}%".format(accuracy_P * 100))

Accuracy on LionData data: 17.00%


In [29]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.4575
Recall: 0.17
F1 Score: 0.15899999999999997


**NyalaDataSet**

In [30]:
# Download dataset (if not already downloaded)
datasets.NyalaData.get_data('../data/NyalaData')
# Load dataset metadata
metadata = datasets.NyalaData('../data/NyalaData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET NyalaData: DOWNLOADING STARTED.


main.zip: 495MB [00:19, 24.8MB/s]


DATASET NyalaData: EXTRACTING STARTED.
DATASET NyalaData: FINISHED.



Epoch 0:   0%|                                                               | 0/16 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  94%|██████████████████████████████████████████████████▋   | 15/16 [00:50<00:03,  3.17s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 16/16 [00:50<00:00,  3.15s/it]


Epoch 1: Average Loss = 37.36736297607422


...

Epoch 18: Average Loss = 32.812957763671875


Epoch 18: 100%|█████████████████████████████████████████████████████| 16/16 [00:50<00:00,  3.13s/it]


Epoch 19: Average Loss = 32.38169860839844


Epoch 19: 100%|█████████████████████████████████████████████████████| 16/16 [00:50<00:00,  3.17s/it]

Epoch 20: Average Loss = 31.984230041503906


In [31]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_NyalaData_retrained_checkpoint.pth")

In [32]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone)
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 15/15 [03:44<00:00, 14.95s/it]


In [33]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[ 0.17458087, -0.12799019, -0.14316124, ...,  0.02647011,
        -0.16366658,  0.3293782 ],
       [ 0.2311159 ,  0.4088794 ,  0.08644719, ...,  0.28991178,
         0.16410036,  0.10325845],
       [ 0.13669765,  0.24152726,  0.20683645, ...,  0.8760791 ,
         0.17001821,  0.18880738],
       ...,
       [ 0.16833487,  0.26378947,  0.18843402, ...,  0.05412264,
         0.35777196, -0.03758318],
       [ 0.22315022, -0.13928305, -0.02781638, ...,  0.3834638 ,
        -0.13022791,  0.41730767],
       [ 0.03482383,  0.28253952,  0.5250732 , ...,  0.05200556,
         0.6731806 , -0.04722582]], dtype=float32)}


In [34]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['19' '10' '57' '149' '56' '102' '147' '201' '114' '71' '267' '51' '33'
 '139' '49' '32' '36' '30' '131' '68' '85' '68' '19' '51' '260' '8' '162'
 '102' '125' '46' '145' '8' '269' '166' '23' '140' '37' '32' '144' '178'
 '181' '250' '56' '46' '8' '122' '114' '69' '268' '149' '10' '147' '55'
 '108' '27' '252' '47' '147' '32' '150' '46' '89' '63' '85' '147' '74'
 '20' '85' '16' '48' '113' '140' '3' '240' '102' '166' '125' '9' '27' '36'
 '47' '145' '23' '85' '89' '32' '17' '62' '6' '69' '3' '125' '42' '217'
 '87' '188' '29' '85' '40' '146']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [35]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on NyalaData data: {:.2f}%".format(accuracy_P * 100))

Accuracy on NyalaData data: 41.00%


In [36]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.7878333333333333
Recall: 0.41
F1 Score: 0.40993650793650793


**StripeSpotter Dataset**

In [37]:
# Download dataset (if not already downloaded)
datasets.StripeSpotter.get_data('../data/StripeSpotter')
# Load dataset metadata
metadata = datasets.StripeSpotter('../data/StripeSpotter')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET StripeSpotter: DOWNLOADING STARTED.


data-20110718.zip: 71.7MB [00:01, 59.0MB/s]                            
data-20110718.z02: 100%|██████████| 78.6M/78.6M [00:01<00:00, 54.9MB/s]
data-20110718.z01: 100%|██████████| 78.6M/78.6M [00:04<00:00, 17.2MB/s]


DATASET StripeSpotter: EXTRACTING STARTED.
 copying: data/
 copying: data/.DS_Store
 copying: data/images/
 copying: data/images/img-0000001.jpg
 copying: data/images/img-0000002.jpg
 copying: data/images/img-0000003.jpg
 copying: data/images/img-0000004.jpg
 copying: data/images/img-0000005.jpg
 copying: data/images/img-0000006.jpg
 copying: data/images/img-0000007.jpg
 copying: data/images/img-0000008.jpg
 copying: data/images/img-0000009.jpg
 copying: data/images/img-0000010.jpg
 copying: data/images/img-0000011.jpg
 copying: data/images/img-0000012.jpg
 copying: data/images/img-0000013.jpg
 copying: data/images/img-0000014.jpg
 copying: data/images/img-0000015.jpg
 copying: data/images/img-0000016.jpg
 copying: data/images/img-0000017.jpg
 copying: data/images/img-0000018.jpg
 copying: data/images/img-0000019.jpg
 copying: data/images/img-0000020.jpg
 copying: data/images/img-0000021.jpg
 copying: data/images/img-0000022.jpg
 copying: data/images/img-0000023.jpg
 copying: data/imag

Epoch 0:   0%|                                                                | 0/7 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  86%|████████████████████████████████████████████████        | 6/7 [00:23<00:03,  3.78s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|████████████████████████████████████████████████████████| 7/7 [00:24<00:00,  3.51s/it]


Epoch 1: Average Loss = 35.83243179321289


...

Epoch 18: Average Loss = 7.033753395080566


Epoch 18: 100%|███████████████████████████████████████████████████████| 7/7 [00:24<00:00,  3.48s/it]


Epoch 19: Average Loss = 6.730792999267578


Epoch 19: 100%|███████████████████████████████████████████████████████| 7/7 [00:24<00:00,  3.45s/it]

Epoch 20: Average Loss = 5.324847221374512


In [38]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_StripeSpotter_retrained_checkpoint.pth")

In [39]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [40]:
# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone)
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|█████████████████████████████████████████████████████████████████| 6/6 [01:31<00:00, 15.32s/it]


In [41]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[ 0.13596988,  0.05231406, -0.09156561, ..., -0.15139128,
        -0.04587702,  0.09240992],
       [-0.13947216,  0.02746603,  0.03109676, ...,  0.825261  ,
         0.03565642,  0.07096192],
       [ 0.23381498, -0.10402443, -0.0431721 , ...,  0.02454687,
        -0.02254226, -0.06648846],
       ...,
       [ 0.1527724 , -0.02696311,  0.04976434, ..., -0.22570018,
        -0.02899821, -0.07573766],
       [ 0.08829218,  0.01991665, -0.02839635, ..., -0.08294424,
         0.0104351 , -0.04776704],
       [ 0.00177036,  0.01424594, -0.0677551 , ...,  0.07499459,
        -0.06589913,  0.02301403]], dtype=float32)}


In [42]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['09_212' 'S09_161' '08_085' '08_100' '09_200' 'OP05_625' '09_212'
 '02_149' '01_579' '08_034' '01_679' '08_078' '01_460' '10_002' '01_136'
 '09_212' '09_200' 'S09_111' '02_149' 'S09_069' '08_100' '01_363' '01_363'
 '08_078' '08_034' '01_805' '09_355' '08_085' 'OP05_546' '09_212' '08_100'
 'S09_088' '09_182' 'S09_215' '02_162' '01_230' '08_085' '01_661'
 '02_1026' '09_185' '01_136' '01_661' '01_661' '02_161' '01_165' '02_161'
 'S09_174' 'S09_067' '08_100' 'OP04_225' '02_149' 'S09_067' '09_200'
 '04_008' '01_661' '09_182' '01_805' '02_161' '10_002' '01_036' '02_149'
 '01_165' '09_212' 'S09_211' 'S09_215' '03_045' '01_579' 'OP04_225'
 '08_100' '08_100' '04_008' '09_212' 'OP05_625' 'OP05_625' '02_149'
 '01_230' '09_200' 'OP05_625' '01_661' 'OP04_225' '01_661' '09_212'
 'S09_174' '01_679' '01_661' 'S09_067' 'S09_111' 'S09_112' '09_212'
 'OP04_225' '02_161' '09_185' '04_008' '08_100' '01_460' '01_679'
 'S09_111' '01_679' '01_036' 'S09_067']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [43]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on StripeSpotter data: {:.2f}%".format(accuracy_P * 100))

Accuracy on StripeSpotter data: 99.00%


In [44]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 0.9925
Recall: 0.99
F1 Score: 0.9857142857142857


**CZoo Dataset**

In [45]:
# Download dataset (if not already downloaded)
datasets.CZoo.get_data('../data/CZoo')
# Load dataset metadata
metadata = datasets.CZoo('../data/CZoo')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET CZoo: DOWNLOADING STARTED.


master.zip: 634MB [00:19, 32.4MB/s]


DATASET CZoo: EXTRACTING STARTED.
DATASET CZoo: FINISHED.



Epoch 0:   0%|                                                               | 0/17 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  94%|██████████████████████████████████████████████████▊   | 16/17 [00:25<00:01,  1.53s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 17/17 [00:26<00:00,  1.54s/it]


Epoch 1: Average Loss = 32.2830696105957


...

Epoch 18: Average Loss = 0.5696266889572144


Epoch 18: 100%|█████████████████████████████████████████████████████| 17/17 [00:26<00:00,  1.57s/it]


Epoch 19: Average Loss = 0.660017192363739


Epoch 19: 100%|█████████████████████████████████████████████████████| 17/17 [00:26<00:00,  1.57s/it]

Epoch 20: Average Loss = 0.5310349464416504


In [46]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_CZoo_retrained_checkpoint.pth")

In [47]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone)
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 16/16 [04:04<00:00, 15.26s/it]


In [48]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[ 9.3994880e-01,  9.5228815e-01,  9.4780552e-01, ...,
        -3.6315462e-03, -1.6923122e-02,  7.4291490e-02],
       [ 9.4823766e-01,  9.6318769e-01,  9.4703448e-01, ...,
         2.7676439e-02, -5.8071986e-03,  5.4770883e-02],
       [ 9.4083500e-01,  9.5213979e-01,  9.4076729e-01, ...,
        -1.2622400e-02, -1.0027090e-02,  6.9663577e-02],
       ...,
       [ 9.5904005e-01,  9.6890342e-01,  9.6100354e-01, ...,
        -1.0044585e-02,  7.6206960e-04,  5.8352157e-02],
       [ 9.6556044e-01,  9.7273988e-01,  9.6762919e-01, ...,
        -1.0882381e-02, -6.3365782e-03,  4.9169149e-02],
       [ 9.5119059e-01,  9.7465503e-01,  9.6168298e-01, ...,
        -2.6328079e-02,  1.0644794e-02,  4.6287410e-02]], dtype=float32)}


In [49]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['Alex' 'Alex' 'Alex' 'Jahaga' 'Jahaga' 'Alex' 'Alex' 'Jahaga' 'Alex'
 'Jahaga' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex'
 'Alex' 'Alex' 'Alex' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Gertrudia' 'Gertrudia' 'Gertrudia' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Annett' 'Annett' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Al

/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [50]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on CZoo data: {:.2f}%".format(accuracy_P * 100))

Accuracy on CZoo data: 100.00%


In [51]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**CowDataset**

In [52]:
# Download dataset (if not already downloaded)
datasets.CowDataset.get_data('../data/CowDataset')
# Load dataset metadata
metadata = datasets.CowDataset('../data/CowDataset')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET CowDataset: DOWNLOADING STARTED.


31210192: 4.45GB [03:37, 20.4MB/s]                            


DATASET CowDataset: EXTRACTING STARTED.
DATASET CowDataset: FINISHED.



Epoch 0:   0%|                                                               | 0/12 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 12/12 [03:10<00:00, 14.05s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 12/12 [03:10<00:00, 15.87s/it]


Epoch 1: Average Loss = 32.46739196777344


...

Epoch 18: Average Loss = 0.36389413475990295


Epoch 18: 100%|█████████████████████████████████████████████████████| 12/12 [03:09<00:00, 15.76s/it]


Epoch 19: Average Loss = 0.26742076873779297


Epoch 19: 100%|█████████████████████████████████████████████████████| 12/12 [03:09<00:00, 15.77s/it]

Epoch 20: Average Loss = 0.3004893362522125


In [53]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_CowDataset_retrained_checkpoint.pth")

In [54]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone)
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 11/11 [04:02<00:00, 22.02s/it]


In [55]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[-7.70535469e-02, -6.34519756e-02, -2.44981144e-02, ...,
         2.08241912e-03,  1.41871557e-01,  1.12791575e-01],
       [-8.94237235e-02, -5.46383634e-02, -1.96823716e-01, ...,
        -2.48982310e-02,  1.25759304e-01,  4.34936136e-02],
       [-3.48647237e-02, -4.10470180e-02, -7.39378929e-02, ...,
         3.15591320e-02,  1.60942078e-01,  1.17216751e-01],
       ...,
       [-9.80513096e-02, -8.39137733e-02, -3.97669673e-02, ...,
        -9.48219560e-04,  1.59420148e-01,  1.33447722e-01],
       [ 1.95122287e-02, -6.51443824e-02,  9.44821909e-02, ...,
        -1.24279052e-01,  1.18369022e-02, -1.11415647e-02],
       [-7.75196999e-02,  1.80972498e-02, -9.42564905e-02, ...,
        -1.26689956e-01,  9.36895907e-01,  9.64226127e-01]], dtype=float32)}


In [56]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)

Predictions for 100 test Images:-
 ['2' '9' '2' '13' '1' '6' '4' '7' '1' '4' '10' '9' '13' '3' '6' '10' '6'
 '4' '2' '13' '6' '2' '2' '9' '8' '8' '4' '7' '2' '12' '6' '6' '2' '4' '5'
 '2' '7' '9' '4' '5' '9' '11' '1' '8' '8' '1' '9' '4' '3' '7' '12' '1' '2'
 '5' '5' '9' '4' '1' '10' '3' '11' '12' '5' '3' '6' '5' '5' '2' '7' '11'
 '8' '5' '3' '9' '8' '7' '2' '1' '11' '1' '6' '3' '12' '6' '3' '8' '10'
 '11' '7' '6' '1' '8' '6' '3' '11' '10' '10' '2' '3' '13']


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T


In [57]:
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on CowDataset data: {:.2f}%".format(accuracy_P * 100))

Accuracy on CowDataset data: 100.00%


In [58]:
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0


 # **On New Unseen Dataset**

**DogFaceNet Dataset**

In [59]:

# Download dataset (if not already downloaded)
datasets.DogFaceNet.get_data('../data/DogFaceNet')

# Load dataset metadata
metadata = datasets.DogFaceNet('../data/DogFaceNet')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=20,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()

DATASET DogFaceNet: DOWNLOADING STARTED.


DogFaceNet_Dataset_224_1.zip: 75.6MB [00:01, 44.7MB/s]                            


DATASET DogFaceNet: EXTRACTING STARTED.
DATASET DogFaceNet: FINISHED.



Epoch 0:   0%|                                                               | 0/66 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0:  98%|█████████████████████████████████████████████████████▏| 65/66 [01:40<00:01,  1.53s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 66/66 [01:40<00:00,  1.53s/it]


Epoch 1: Average Loss = 39.363136291503906


...

Epoch 18: Average Loss = 28.167173385620117


Epoch 18: 100%|█████████████████████████████████████████████████████| 66/66 [01:41<00:00,  1.54s/it]


Epoch 19: Average Loss = 27.381820678710938


Epoch 19: 100%|█████████████████████████████████████████████████████| 66/66 [01:41<00:00,  1.54s/it]

Epoch 20: Average Loss = 26.730960845947266


In [60]:
import torch
trainer.save("retrained_chks", file_name="arcfaceloass_DogFaceNet_retrained_checkpoint.pth")

In [61]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [62]:
# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model)
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

100%|███████████████████████████████████████████████████████████████| 65/65 [17:24<00:00, 16.07s/it]


In [63]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[-1.31476671e-04, -1.31595582e-02,  5.87870516e-02, ...,
         1.36885494e-01,  9.90640074e-02,  5.95364906e-03],
       [-5.12946919e-02, -3.94692943e-02,  7.69007280e-02, ...,
        -4.48677130e-02,  3.03032696e-02, -9.66164656e-03],
       [ 2.79198941e-02, -1.38392240e-01,  1.39549002e-01, ...,
        -3.72362435e-02,  5.22092059e-02,  8.85126293e-02],
       ...,
       [ 1.17416896e-01, -3.67560983e-02,  1.10490233e-01, ...,
        -3.28138322e-02,  5.35747185e-02, -3.89259383e-02],
       [ 2.41332036e-02,  1.33320227e-01,  7.91649073e-02, ...,
        -7.09844660e-03,  2.40814500e-02,  6.20542690e-02],
       [-3.73712480e-02, -1.73014719e-02, -5.10162003e-02, ...,
        -2.04134341e-02,  3.54679190e-02, -5.23208007e-02]], dtype=float32)}


In [64]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on IPanda50 data: {:.2f}%".format(accuracy_P * 100))
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Predictions for 100 test Images:-
 ['426' '182' '919' '1122' '428' '378' '1063' '759' '470' '405' '764' '448'
 '641' '1339' '432' '673' '1075' '274' '296' '234' '790' '991' '485' '425'
 '280' '407' '324' '1248' '1232' '896' '892' '448' '240' '251' '448' '930'
 '223' '210' '345' '763' '622' '1324' '158' '127' '313' '1254' '1139'
 '208' '368' '424' '293' '415' '825' '147' '373' '192' '333' '333' '1314'
 '957' '329' '89' '189' '152' '122' '631' '367' '646' '26' '1368' '878'
 '673' '174' '474' '1169' '458' '473' '149' '373' '622' '540' '919' '589'
 '913' '782' '1357' '361' '88' '1324' '203' '183' '200' '299' '191' '397'
 '187' '270' '563' '1020' '1074']
Accuracy on IPanda50 data: 90.00%
Precision: 0.9833333333333334
Recall: 0.9
F1 Score: 0.8993333333333333


/opt/conda/lib/python3.10/site-packages/wildlife_tools/inference/classifier.py:61: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  results = pd.DataFrame(results).T.fillna(method="ffill").T
